In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
from scipy.spatial import distance

def calculate_credit_score(financial_ratio, industry_thresholds, expert_thresholds, inverse_relationship=False):
    """
    Calculate the credit score, rating, confidence level, and range scores based on a single financial ratio and thresholds.

    Args:
        financial_ratio (float): Financial ratio value.
        industry_thresholds (dict): Dictionary of industry thresholds for each rating.
        expert_thresholds (dict): Dictionary of expert thresholds for each rating.
        inverse_relationship (bool): Indicates if the financial ratio has an inverse relationship (lower is better).

    Returns:
        tuple: A tuple containing the credit score (int), credit rating (str),
               confidence level (float), rating scales (dict), and range scores (dict).
    """
    if inverse_relationship:
        financial_ratio = 1 / financial_ratio

    distances = {
        rating: distance.euclidean(
            [financial_ratio],
            [(industry_thresholds[rating] + expert_thresholds[rating]) / 2]
        )
        for rating in ['good', 'fair', 'poor']
    }
    credit_rating = min(distances, key=distances.get)
    
    max_score = 100
    min_score = 1
    min_threshold = min(industry_thresholds.values())
    max_threshold = max(industry_thresholds.values())

    credit_score = (financial_ratio - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score
    credit_score = round(credit_score)
    
    confidence_level = 1 - distances[credit_rating] / (max_threshold - min_threshold) if credit_score < max_score else 1.0


    range_scores = {
        'good': {
            'min': (industry_thresholds['good'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score,
            'max': max_score
        },
        'fair': {
            'min': (industry_thresholds['fair'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score,
            'max': (industry_thresholds['good'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score - 1
        },
        'poor': {
            'min': min_score,
            'max': (industry_thresholds['fair'] - min_threshold) / (max_threshold - min_threshold) * (max_score - min_score) + min_score - 1
        }
    }

    return credit_score, credit_rating, confidence_level, range_scores


def get_credit_recommendation(credit_score, credit_rating, confidence_level):
    """
    Generate a credit recommendation based on the credit score, rating, and confidence level.

    Args:
        credit_score (int): Credit score value.
        credit_rating (str): Credit rating category.
        confidence_level (float): Confidence level of the credit rating.

    Returns:
        str: Credit recommendation message.
    """
    if credit_score >= 80:
        risk_level = "low"
    elif credit_score >= 60:
        risk_level = "medium"
    else:
        risk_level = "high"

    recommendation = (
        f"The entity has a {credit_rating} credit rating with a confidence level of {confidence_level:.2%}. "
        f"The credit score of {credit_score} indicates a {risk_level} risk level."
    )

    return recommendation

In [154]:
company_ratios = {
    'A': [0.2, 0.4, 1.5, 0.6, 1.2],
    'B': [0.9, 1.1, 1.3, 0.7, 0.8],
    'C': [1.0, 0.6, 0.1, 0.9, 0.2]
}

# Define industry and expert thresholds for each financial ratio
industry_thresholds = [
    {'good': 1.0, 'fair': 0.7, 'poor': 0.4},
    {'good': 0.6, 'fair': 0.4, 'poor': 0.2},
    {'good': 1.2, 'fair': 0.9, 'poor': 0.6},
    {'good': 0.8, 'fair': 0.6, 'poor': 0.4},
    {'good': 1.1, 'fair': 0.8, 'poor': 0.5}
]

expert_thresholds = [
    {'good': 1.2, 'fair': 0.8, 'poor': 0.5},
    {'good': 0.7, 'fair': 0.5, 'poor': 0.3},
    {'good': 1.4, 'fair': 1.0, 'poor': 0.7},
    {'good': 0.9, 'fair': 0.7, 'poor': 0.5},
    {'good': 1.3, 'fair': 0.9, 'poor': 0.6}
]

# Define inverse relationship for each financial ratio
inverse_relationships = [False, True, False, True, False]

In [155]:
#     # Calculate distances using the average of industry and expert thresholds
#     distances = {
#         rating: abs(financial_ratio - ((industry_thresholds[rating] + expert_thresholds[rating]) / 2))
#         for rating in ['good', 'fair', 'poor']
#     }

In [180]:
def calculate_overall_credit_score(credit_scores):
    """
    Calculate the overall credit score and rating based on individual financial ratio credit scores.

    Args:
        credit_scores (list): List of credit scores for each financial ratio.

    Returns:
        tuple: A tuple containing the overall credit score (float) and overall credit rating (str).
    """
    
    max_score = 100
    min_score = 0
    
    overall_credit_score = sum(credit_scores) / len(credit_scores)
    
    # Rescale the overall credit score to have a maximum rating of 100
    rescaled_credit_score = (overall_credit_score - min_score) / (max_score - min_score) * 100
    
    if overall_credit_score >= 75:
        overall_credit_rating = "Good"
    elif overall_credit_score >= 50:
        overall_credit_rating = "Fair"
    else:
        overall_credit_rating = "Poor"
    
    return overall_credit_score, overall_credit_rating

def calculate_credit_score(financial_ratio, industry_thresholds, expert_thresholds, inverse_relationship=False):
    """
    Calculate the actual credit score within a universal score range based on where the financial ratio
    falls within its thresholds, alongside the credit rating, confidence level, and universal range scores.
    
    Args:
        financial_ratio (float): Financial ratio value.
        industry_thresholds (dict): Dictionary of industry thresholds for each rating.
        expert_thresholds (dict): Dictionary of expert thresholds for each rating.
        inverse_relationship (bool): Indicates if the financial ratio has an inverse relationship (lower is better).

    Returns:
        tuple: A tuple containing the actual credit score (int), credit rating (str),
               confidence level (float), and universal score ranges (dict).
    """
    # Adjust thresholds and financial ratio for inverse relationships
    if inverse_relationship:
        financial_ratio = 1 / financial_ratio
        industry_thresholds = {k: 1/v for k, v in industry_thresholds.items()}
        expert_thresholds = {k: 1/v for k, v in expert_thresholds.items()}
    
    distances = {
        rating: distance.euclidean(
            [financial_ratio],
            [(industry_thresholds[rating] + expert_thresholds[rating]) / 2])
        for rating in ['good', 'fair', 'poor']
    }
    
    credit_rating = min(distances, key=distances.get)
    
    # Universal score ranges for each rating category
    score_ranges = {
        'good': (75, 100),
        'fair': (50, 74),
        'poor': (0, 49)
    }

    # Determine the min and max threshold for the financial ratio
    min_threshold = min(industry_thresholds.values())
    max_threshold = max(industry_thresholds.values())
    
    # Calculate where the financial ratio falls within its thresholds
    financial_ratio = max(min_threshold, min(financial_ratio, max_threshold))
    ratio_position = (financial_ratio - min_threshold) / (max_threshold - min_threshold)
    
    # Apply this proportion to the corresponding universal score range
    credit_score_range = score_ranges[credit_rating]
    score_range_width = credit_score_range[1] - credit_score_range[0]
    credit_score = credit_score_range[0] + ratio_position * score_range_width
    
    # Calculate confidence level
    confidence_level = (credit_score - credit_score_range[0]) / score_range_width
    confidence_level = max(0, min(1, confidence_level))
    
    return credit_score, credit_rating, confidence_level, score_ranges

In [184]:
# Iterate over each company and calculate credit scores for each financial ratio
for company, ratios in company_ratios.items():
    print(f"Company {company}:")
    credit_scores = []

    for i, ratio in enumerate(ratios):
        credit_score, credit_rating, confidence_level, range_scores = calculate_credit_score(
            ratio, industry_thresholds[i], expert_thresholds[i], inverse_relationships[i]
        )
        recommendation = get_credit_recommendation(credit_score, credit_rating, confidence_level)

        credit_scores.append(credit_score)

        print(f"  Financial Ratio {i+1}:")
        print(f"    Credit Score: {credit_score}")
        print(f"    Credit Rating: {credit_rating}")
        print(f"    Confidence Level: {confidence_level:.2%}")
        print(f"    Range Scores:")
        for rating, scores in range_scores.items():
            print(f"      {rating.capitalize()}: {scores[0]:.2f} - {scores[1]:.2f}")
        print(f"    Recommendation: {recommendation}")
        
        #break

    overall_credit_score, overall_credit_rating = calculate_overall_credit_score(credit_scores)
    print(f"  Overall Credit Score: {overall_credit_score:.2f}")
    print(f"  Overall Credit Rating: {overall_credit_rating}")
    print()
    

Company A:
  Financial Ratio 1:
    Credit Score: 0.0
    Credit Rating: poor
    Confidence Level: 0.00%
    Range Scores:
      Good: 75.00 - 100.00
      Fair: 50.00 - 74.00
      Poor: 0.00 - 49.00
    Recommendation: The entity has a poor credit rating with a confidence level of 0.00%. The credit score of 0.0 indicates a high risk level.
  Financial Ratio 2:
    Credit Score: 56.0
    Credit Rating: fair
    Confidence Level: 25.00%
    Range Scores:
      Good: 75.00 - 100.00
      Fair: 50.00 - 74.00
      Poor: 0.00 - 49.00
    Recommendation: The entity has a fair credit rating with a confidence level of 25.00%. The credit score of 56.0 indicates a high risk level.
  Financial Ratio 3:
    Credit Score: 100.0
    Credit Rating: good
    Confidence Level: 100.00%
    Range Scores:
      Good: 75.00 - 100.00
      Fair: 50.00 - 74.00
      Poor: 0.00 - 49.00
    Recommendation: The entity has a good credit rating with a confidence level of 100.00%. The credit score of 100.0 indi

In [172]:
def calculate_confidence(credit_score, credit_range):
    lower, upper = credit_range
    return (upper - credit_score) / (upper - lower) * 100

credit_score = 56.0
credit_range = (50.00, 74.00)
confidence = calculate_confidence(credit_score, credit_range)
print(f"Confidence Level: {confidence:.2f}%")


Confidence Level: 75.00%


In [173]:
74 - 50

24

In [174]:
56 - 50

6

In [175]:
6 / 24

0.25

In [116]:
ratio, industry_thresholds[i], expert_thresholds[i], inverse_relationships[i]

(1.2,
 {'good': 1.1, 'fair': 0.8, 'poor': 0.5},
 {'good': 1.3, 'fair': 0.9, 'poor': 0.6},
 False)

In [ ]:
def calculate_overall_credit_score(credit_scores):
    """
    Calculate the overall credit score and rating based on individual financial ratio credit scores.

    Args:
        credit_scores (list): List of credit scores for each financial ratio.

    Returns:
        tuple: A tuple containing the overall credit score (float) and overall credit rating (str).
    """
    
    max_score = 100
    min_score = 0
    
    overall_credit_score = sum(credit_scores) / len(credit_scores)
    
    # Rescale the overall credit score to have a maximum rating of 100
    rescaled_credit_score = (overall_credit_score - min_score) / (max_score - min_score) * 100
    
    if overall_credit_score >= 75:
        overall_credit_rating = "Good"
    elif overall_credit_score >= 50:
        overall_credit_rating = "Fair"
    else:
        overall_credit_rating = "Poor"
    
    return overall_credit_score, overall_credit_rating

def calculate_credit_score(financial_ratio, industry_thresholds, expert_thresholds, inverse_relationship=False):
    """
    Calculate the actual credit score within a universal score range based on where the financial ratio
    falls within its thresholds, alongside the credit rating, confidence level, and universal range scores.
    
    Args:
        financial_ratio (float): Financial ratio value.
        industry_thresholds (dict): Dictionary of industry thresholds for each rating.
        expert_thresholds (dict): Dictionary of expert thresholds for each rating.
        inverse_relationship (bool): Indicates if the financial ratio has an inverse relationship (lower is better).

    Returns:
        tuple: A tuple containing the actual credit score (int), credit rating (str),
               confidence level (float), and universal score ranges (dict).
    """
    # Adjust thresholds and financial ratio for inverse relationships
    if inverse_relationship:
        financial_ratio = 1 / financial_ratio
        industry_thresholds = {k: 1/v for k, v in industry_thresholds.items()}
        expert_thresholds = {k: 1/v for k, v in expert_thresholds.items()}
    
    distances = {
        rating: distance.euclidean(
            [financial_ratio],
            [(industry_thresholds[rating] + expert_thresholds[rating]) / 2])
        for rating in ['good', 'fair', 'poor']
    }
    
    credit_rating = min(distances, key=distances.get)
    
    # Universal score ranges for each rating category
    score_ranges = {
        'good': (75, 100),
        'fair': (50, 74),
        'poor': (0, 49)
    }

    # Determine the min and max threshold for the financial ratio
    min_threshold = min(industry_thresholds.values())
    max_threshold = max(industry_thresholds.values())
    
    # Calculate where the financial ratio falls within its thresholds
    financial_ratio = max(min_threshold, min(financial_ratio, max_threshold))
    ratio_position = (financial_ratio - min_threshold) / (max_threshold - min_threshold)
    
    # Apply this proportion to the corresponding universal score range
    credit_score_range = score_ranges[credit_rating]
    score_range_width = credit_score_range[1] - credit_score_range[0]
    credit_score = credit_score_range[0] + ratio_position * score_range_width
    
    # Calculate confidence level
    confidence_level = (credit_score - credit_score_range[0]) / score_range_width
    confidence_level = max(0, min(1, confidence_level))
    
    return credit_score, credit_rating, confidence_level, score_ranges

In [186]:
# Iterate over each company and calculate credit scores for each financial ratio
for company, ratios in company_ratios.items():
    print(f"Company {company}:")
    credit_scores = []

    for i, ratio in enumerate(ratios):
        credit_score, credit_rating, confidence_level, range_scores = calculate_credit_score(
            ratio, industry_thresholds[i], expert_thresholds[i], inverse_relationships[i]
        )
        recommendation = get_credit_recommendation(credit_score, credit_rating, confidence_level)

        credit_scores.append(credit_score)

    overall_credit_score, overall_credit_rating = calculate_overall_credit_score(credit_scores)
    print(f"Overall Credit Score: {overall_credit_score:.2f}")
    print(f"Overall Credit Rating: {overall_credit_rating}")
    print()
    

Company A:
Overall Credit Score: 62.80
Overall Credit Rating: Fair

Company B:
Overall Credit Score: 72.09
Overall Credit Rating: Fair

Company C:
Overall Credit Score: 50.00
Overall Credit Rating: Fair

